In [1]:
from __future__ import print_function
import h5py
from numpy import *
from matplotlib.pyplot import *
import torch
import torch.utils.data
from torch import nn, optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import os
from torchinfo import summary

/data1/jcampos/.conda/envs/ml4qick-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data1/jcampos/.conda/envs/ml4qick-env/lib/python3.7/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from model import Classifier 
from hawq_model import HawqClassifier 
from brevitas_model import BrevitasModel 

In [4]:
base_model = Classifier()
base_model.load_state_dict(torch.load("../checkpoints/checkpoint_tiny_affine.pth"))

# model = HawqClassifier(base_model)
model = BrevitasModel()

In [5]:
# Hyper-Parameters
torch.manual_seed(4)
epochs = 210
train = 9000#data.shape[0]*0.99
learning_rate = 1e-4

device = torch.device("cpu")
kwargs = {'num_workers':0, 'pin_memory': True} 
plot_interval = 1

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Loss tracks
train_loss_track=np.array([])
test_loss_track=np.array([])
acc_track=np.array([])

In [7]:
# Load the raw data
IF = -136.75/1e3
with h5py.File(r'../../datasets/qubits/00002_IQ_plot_raw.h5', 'r') as f:
    adc_g_1 = array(f['adc_g_1'])[0]
    adc_g_2 = array(f['adc_g_2'])[0]
    adc_e_1 = array(f['adc_e_1'])[0]
    adc_e_2 = array(f['adc_e_2'])[0]

""" Select the range of time series data. Each data is 2000 element vector 
representing 2000ns readout signal"""
csr = range(500,1500)
sr = len(csr)

I_g = adc_g_1[:,csr]
Q_g = adc_g_2[:,csr] 
I_e = adc_e_1[:,csr] 
Q_e = adc_e_2[:,csr] 

# Dataset Creation
data = zeros((adc_g_1.shape[0]*2,sr,2))
data[0:adc_g_1.shape[0],:,0] = I_g
data[0:adc_g_1.shape[0],:,1] = Q_g
data[adc_g_1.shape[0]:adc_g_1.shape[0]*2,:,0] = I_e
data[adc_g_1.shape[0]:adc_g_1.shape[0]*2,:,1] = Q_e

labels = zeros(I_e.shape[0]*2)
labels[I_e.shape[0]:I_e.shape[0]*2] = 1

data = torch.from_numpy(data).float()
labels = torch.from_numpy(labels).float()

class Qubit_Readout_Dataset():
    
    def __init__(self):
        self.data = data
        self.labels = labels    
        self.data = self.data.reshape(len(data),sr*2)
       
    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Hyper-Parameters
torch.manual_seed(4)
epochs = 210
train = 9000#data.shape[0]*0.99
test = len(data)-train
batch_size = 12800
learning_rate = 1e-4

kwargs = {'num_workers':0, 'pin_memory': True} 
plot_interval = 1

# Dataloader Prep
dataset = Qubit_Readout_Dataset()
train_data, test_data = torch.utils.data.random_split(dataset, [int(train), int(test)])

num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                            num_workers = num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, 
                                            num_workers = num_workers, shuffle=True)


In [8]:
# Training loop
for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    model.train() 
    
    for data, labels in train_loader:
        optimizer.zero_grad()
        data, labels = data.to(device), labels.to(device)
        states = model(data)
        loss = criterion(states, labels.long())
        loss.backward()
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        
    train_loss_track = np.append(train_loss_track,np.asarray(train_loss))
        
    test_loss = 0    
    model.eval()
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            print(data.shape)
            states = model(data)
            loss = criterion(states, labels.long())
            test_loss += loss.detach().cpu().numpy()
    
    test_loss_track = np.append(test_loss_track,np.asarray(test_loss))
    
    if epoch % plot_interval == 0:
        print('====> Epoch: {} Training loss: {:.6f}'.format(
                  epoch, train_loss ))
        print('====> Epoch: {} Test loss: {:.6f}'.format(
                  epoch, test_loss ))

        p1, p2 = 0, 800
        # plt.plot(train_loss_track[p1:p2],label = 'Training')
        # plt.plot(test_loss_track[p1:p2],label='Testing')
        # plt.legend()
        # plt.xlabel('Epochs')
        # plt.title("Training - Test Loss")
        # plt.figure(figsize = (12,7))
        # plt.show()
        # plt.close()

    # Readout Fidelity
    model.eval() 
    cc = 0
    y_true = torch.tensor([]).to(device)
    y_pred = torch.tensor([]).to(device)
    
    with torch.no_grad():
        for data, target in test_loader:
            
            data=data.to(device)
            states = model(data)
            target = target.to(device) 
    
            val, ind = torch.max(states,1)
            y_pred = torch.cat((y_pred, ind), 0)
            y_true = torch.cat((y_true, target), 0)
    
    acc = y_true-y_pred
    accuracy = (len(y_true)-torch.count_nonzero(acc))/len(y_true)
    accuracy = accuracy.item()
    
    acc_track = np.append(acc_track,np.asarray(accuracy))
    print('Readout Fidelity: %', accuracy*100)


  0%|          | 1/210 [00:00<02:35,  1.34it/s]

torch.Size([1000, 2000])
====> Epoch: 0 Training loss: 0.714233
====> Epoch: 0 Test loss: 0.716794
Readout Fidelity: % 54.100000858306885


  1%|          | 2/210 [00:01<02:20,  1.48it/s]

torch.Size([1000, 2000])
====> Epoch: 1 Training loss: 0.683939
====> Epoch: 1 Test loss: 0.676656
Readout Fidelity: % 59.10000205039978


  1%|▏         | 3/210 [00:02<02:48,  1.23it/s]

...

torch.Size([1000, 2000])
====> Epoch: 207 Training loss: 0.078697
====> Epoch: 207 Test loss: 0.377238
Readout Fidelity: % 90.10000228881836


100%|█████████▉| 209/210 [02:50<00:00,  1.37it/s]

torch.Size([1000, 2000])
====> Epoch: 208 Training loss: 0.078558
====> Epoch: 208 Test loss: 0.377753
Readout Fidelity: % 90.10000228881836


100%|██████████| 210/210 [02:51<00:00,  1.23it/s]

torch.Size([1000, 2000])
====> Epoch: 209 Training loss: 0.078377
====> Epoch: 209 Test loss: 0.378010
Readout Fidelity: % 90.10000228881836


In [9]:
torch.save(model.state_dict(), 'checkpoints/ckp_brevitas_tiny.pth') 